In [0]:
import sys
import os
import importlib.util
from src.common.utils import to_struct_type

# Añade la ruta absoluta de src/ al sys.path
repo_root = os.path.abspath(os.path.join(os.getcwd(), "../../../.."))  # Ajusta si es necesario
src_path = os.path.join(repo_root, "src")
if src_path not in sys.path:
    sys.path.append(src_path)

# Ahora sí puedes importar de common, etc.

# 1. Ubica el path absoluto a la carpeta tables
tables_path = os.path.abspath(os.path.join(os.getcwd(), "../tables"))

tables_infos = []

for fname in os.listdir(tables_path):
    if fname.endswith(".py") and not fname.startswith("__"):
        fpath = os.path.join(tables_path, fname)
        module_name = fname[:-3]
        spec = importlib.util.spec_from_file_location(module_name, fpath)
        module = importlib.util.module_from_spec(spec)
        spec.loader.exec_module(module)
        if hasattr(module, "get_table_info"):
            info = module.get_table_info()
            tables_infos.append(info)
            print(f"Procesando tabla: {info['table']}")
            print("Path:", info["path"])
            print("Schema:", info["schema"])


In [0]:
for info in tables_infos:
    #schema = to_struct_type(info["schema"])
    spark.sql(f"""
        CREATE TABLE IF NOT EXISTS {info['table']}
        USING DELTA
        LOCATION '{info['path']}'
    """)
